In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 1
setup_one_gpu(GPU)

Picking GPU 1


In [2]:
import sys
import numpy as np
import os.path as osp
from tf_lab.iclr.evaluator import Evaluator
from general_tools.in_out.basics import files_in_subdirs
from tf_lab.iclr.helper import stored_synthetic_samples, top_evaluation_dir

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [16]:
do_jsd = False
top_out_dir = top_evaluation_dir
boost_samples = 3
special_tag = ''
skip = ['test', 'train']

In [15]:
top_syn_dir = '/orions4-zfs/projects/optas/DATA/OUT/iclr/synthetic_samples/'

In [17]:
all_classes = ['airplane', 'car', 'table', 'sofa', 'rifle']
# all_classes = ['chair']
best_model = {'gmm': 'bneck_128_full_32_gaussians', 'l_w_gan': 2000}
generators_zoo = ['gmm']

In [7]:
gan_tag = '_mlp_with_split_1pc_usampled_bnorm_on_encoder_only_emd_bneck_128_lgan_arch'

In [28]:
do_cov = True
do_mmd = True
do_jsd = False
n_reps = 3
mmd_loss = 'emd'
sample_metrics = False

In [29]:
for class_name in all_classes:    
    print class_name
    
    evaluator = Evaluator(class_name)
    evaluator.load_gt_data()
    
    if do_jsd:
        evaluator.prepare_gt_grid_variables()    
        
    for generator_name in generators_zoo:
        jsd_scores = []
        mmd_scores = []
        cov_scores = []
        
        sample_files = []
        if 'gan' in generator_name:
            top_sample_dir = osp.join(top_syn_dir, generator_name, generator_name + '_' + class_name + gan_tag)
            epoch = best_model[generator_name]
            sample_files.append(osp.join(top_sample_dir, 'epoch_%d.npz' % (epoch,) ))
        elif 'gmm' in generator_name:
            gaussian_type = best_model[generator_name]
            sample_files.append(osp.join(top_syn_dir, 'gmm', 'gmm_emd_'+  class_name, gaussian_type + '.npz'))
        else:
            assert False
        
        for _ in range(n_reps):
            print generator_name            
            
            for sample_file in sample_files:
                evaluator.prepare_sample_data(sample_file, boost_samples)
                sample_name = osp.basename(sample_file)[:-len('.npz')]
                print sample_name

            if do_mmd:        
                score = evaluator.compute_mmd(loss=mmd_loss, sample_estimator=sample_metrics, skip=skip)
                mmd_scores.append(score)

            if do_cov:
                score = evaluator.compute_coverage(loss=mmd_loss, sample_estimator=sample_metrics, skip=skip)
                cov_scores.append(score)
            
        print  
        for score_type in [mmd_scores, cov_scores]:
            if len(score_type) > 0:
                total = 0.0
                for s in score_type:
                    total += np.mean(s['val'])
                print total / len(score_type)
        print

airplane
gmm
bneck_128_full_32_gaussians
val 79.4629 34.5885
val 0.70297029703 0.0
gmm
bneck_128_full_32_gaussians
val 78.8543 35.0475
val 0.69801980198 0.0
gmm
bneck_128_full_32_gaussians
val 79.505 35.4845
val 0.688118811881 0.0

79.2740910848
0.696369636964

car
gmm
bneck_128_full_32_gaussians
val 85.4422 20.0956
val 0.668449197861 0.0
gmm
bneck_128_full_32_gaussians
val 85.9136 20.7015
val 0.652406417112 0.0
gmm
bneck_128_full_32_gaussians
val 85.4884 20.4026
val 0.639037433155 0.0

85.6147537231
0.653297682709

table
gmm
bneck_128_full_32_gaussians
val 126.672 36.5819
val 0.701176470588 0.0
gmm
bneck_128_full_32_gaussians
val 125.508 36.2578
val 0.734117647059 0.0
gmm
bneck_128_full_32_gaussians
val 126.146 36.3695
val 0.698823529412 0.0

126.109006246
0.71137254902

sofa
gmm
bneck_128_full_32_gaussians
val 113.864 31.116
val 0.677215189873 0.0
gmm
bneck_128_full_32_gaussians
val 113.367 31.8648
val 0.645569620253 0.0
gmm
bneck_128_full_32_gaussians
val 113.199 28.9478
val 0.67721